In [1]:
#Customers who left are -1, current customers are 1
#Males are 0, females are 1


#binoy says:
#try out imbalanced random forest FROM IMBLEARN

#add github project overview


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

#from stacked_generalizer import StackedGeneralizer           #CANT GET THIS TO WORK WITH PYTHON3, WOULD NEED TO FIX SOURCE CODE MYSELF
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC


In [3]:

starter_df = pd.read_csv(Path("c://users/ajcth/documents/github/bank_churn_project/Resources/BankChurners2.csv"))

starter_df = starter_df[starter_df != 'Unknown'].dropna()
starter_df.sample(5)


,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
8449,711944733,Existing Customer,45,F,2,High School,Married,Less than $40K,Blue,36,...,1457.0,828,629.0,0.852,5401,93,0.661,0.568,0.000111,0.999890
5725,771079833,Existing Customer,40,F,3,High School,Single,Less than $40K,Blue,27,...,1438.3,0,1438.3,0.845,4332,78,0.696,0.000,0.000305,0.999690
2146,717348258,Existing Customer,33,M,1,Graduate,Married,$120K +,Blue,21,...,4459.0,1899,2560.0,1.349,2502,48,1.000,0.426,0.000087,0.999910
9382,709075533,Attrited Customer,41,M,5,Uneducated,Single,$80K - $120K,Blue,32,...,12094.0,0,12094.0,0.995,8607,90,0.957,0.000,0.996900,0.003103
5542,711267633,Attrited Customer,50,F,3,Graduate,Divorced,Less than $40K,Blue,40,...,4248.0,316,3932.0,0.661,2271,36,0.333,0.074,0.998460,0.001542


In [4]:
#Shorten column names so they display better
replacement_column_names = {'Attrition_Flag': 'Status',
        'Customer_Age': 'Age',
        'Education_Level': 'Education',
        'Dependent_count': 'Dependents',
        'Income_Category': 'Income',
        'Card_Category': 'Card Type',
        'Months_on_book': 'Tenure(month)'
        }
starter_df.rename(columns=replacement_column_names, inplace=True)

In [5]:
#Remove columns that are not understood or irrelevant 

starter_df.drop(columns=['Total_Relationship_Count', #¿que?
'Contacts_Count_12_mon', #Not a clue
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', #Some kind of wizardry, incantations
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', #I didnt make this and idk what it do or how it do it
'Total_Amt_Chng_Q4_Q1', 'Total_Ct_Chng_Q4_Q1', #Same as it ever was
'CLIENTNUM' #unnecessary
],
inplace=True
)


In [6]:
#Replace card categories from Blue/Silver/Gold/Plat to 0/1/2/3/4
starter_df = starter_df.replace({'Card Type' : {'Blue': 0, 'Silver': 1, 'Gold' : 3, 'Platinum' : 4}})
starter_df.loc[ : ,['Card Type']] = starter_df.loc[ : ,['Card Type']].astype(int)

#Replace M/F with 0,1
starter_df = starter_df.replace({'Gender' : {'M': 0, 'F': 1}})
starter_df.Gender = starter_df.Gender.astype(int)
#NOT BEING USED CURRENTLY, Replaces M/F with -1/1
#starter_df = starter_df.replace({'Gender' : {'M': '-1', 'F': '1'}})

#Replace customer's status to -1 if they left and 1 if they stayed
#starter_df = starter_df.replace({'Status' : {'Attrited Customer': -1, 'Existing Customer': 1}})
#starter_df.Status = starter_df.Status.astype(int)

#Replace customer's status to -1 if they left and 1 if they stayed
#Another feature option not being used
starter_df = starter_df.replace({'Status' : {'Attrited Customer': -1, 'Existing Customer': 1}})



In [7]:
#binoy says dont use this


"""
use feature creation to make a ratio between dependents and income sources
consider someone divorced as Single. only 10% of divorced people receive alimony payments
        either a 1 (single/divorced) or a 2 (married)
        divide that by the amount of dependents plus one, which represents how many people you take care of
                i.e. a single person with no dependents only takes care of themself, so they are a 1. 
                """
'''
starter_df = starter_df.replace({'Marital_Status' : {'Divorced': 1, 'Single': 1, 'Married' : 2}})
starter_df['MaritalDependentRatio'] = ((starter_df['Marital_Status'] / (starter_df['Dependents'] + 1)) / 2).round(2)
starter_df.drop(columns=['Marital_Status', 'Dependents'], inplace=True)
'''



"\nstarter_df = starter_df.replace({'Marital_Status' : {'Divorced': 1, 'Single': 1, 'Married' : 2}})\nstarter_df['MaritalDependentRatio'] = ((starter_df['Marital_Status'] / (starter_df['Dependents'] + 1)) / 2).round(2)\nstarter_df.drop(columns=['Marital_Status', 'Dependents'], inplace=True)\n"

In [8]:
#OneHotEncode the categorical variable 'Marital Status'
enc = OneHotEncoder(sparse=False)
categorical_features = ["Marital_Status"]
encoded_data = enc.fit_transform(starter_df[categorical_features])
encoded_df = pd.DataFrame(encoded_data, columns = enc.get_feature_names(categorical_features))

#Add the encoded data DF to the original DF
starter_df = pd.concat([encoded_df, starter_df], axis=1)
starter_df = starter_df.dropna()
starter_df.drop(columns='Marital_Status', inplace=True)
starter_df.Marital_Status_Divorced, starter_df.Marital_Status_Married, starter_df.Marital_Status_Single = starter_df.Marital_Status_Divorced.astype(int), starter_df.Marital_Status_Married.astype(int), starter_df.Marital_Status_Single.astype(int)
starter_df.sample(5)

c:\python_files\Anaconda3\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Status,Age,Gender,Dependents,Education,Income,Card Type,Tenure(month),Months_Inactive_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Trans_Amt,Total_Trans_Ct,Avg_Utilization_Ratio
1291,0,1,0,1.0,38.0,1.0,2.0,Uneducated,Less than $40K,0.0,29.0,3.0,3590.0,1924.0,1666.0,1472.0,37.0,0.536
2470,1,0,0,1.0,33.0,1.0,3.0,College,$40K - $60K,0.0,26.0,3.0,6316.0,1344.0,4972.0,2401.0,71.0,0.213
5199,0,1,0,1.0,44.0,1.0,3.0,Graduate,Less than $40K,0.0,27.0,1.0,1480.0,1017.0,463.0,5245.0,80.0,0.687
6722,0,0,1,1.0,40.0,1.0,2.0,High School,Less than $40K,0.0,34.0,2.0,2395.0,1437.0,958.0,4481.0,82.0,0.600
6151,0,1,0,-1.0,54.0,1.0,2.0,Doctorate,Less than $40K,0.0,42.0,4.0,1694.0,0.0,1694.0,1930.0,41.0,0.000


In [9]:
#use feature creation to make a ratio between avg open to buy and credit limit

starter_df['CreditUsage'] = (starter_df['Avg_Open_To_Buy'] / starter_df['Credit_Limit']).round(2)
starter_df.drop(columns=['Avg_Open_To_Buy', 'Credit_Limit'], inplace=True)
starter_df = starter_df.dropna()


In [10]:
starter_df

,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Status,Age,Gender,Dependents,Education,Income,Card Type,Tenure(month),Months_Inactive_12_mon,Total_Revolving_Bal,Total_Trans_Amt,Total_Trans_Ct,Avg_Utilization_Ratio,CreditUsage
0,0,1,0,1.0,45.0,0.0,3.0,High School,$60K - $80K,0.0,39.0,1.0,777.0,1144.0,42.0,0.061,0.94
1,0,0,1,1.0,49.0,1.0,5.0,Graduate,Less than $40K,0.0,44.0,1.0,864.0,1291.0,33.0,0.105,0.90
2,0,1,0,1.0,51.0,0.0,3.0,Graduate,$80K - $120K,0.0,36.0,1.0,0.0,1887.0,20.0,0.000,1.00
4,0,1,0,1.0,40.0,0.0,3.0,Uneducated,$60K - $80K,0.0,21.0,1.0,0.0,816.0,28.0,0.000,1.00
5,0,0,1,1.0,44.0,0.0,2.0,Graduate,$40K - $60K,0.0,36.0,1.0,1247.0,1088.0,24.0,0.311,0.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7076,0,0,1,1.0,38.0,1.0,2.0,High School,$40K - $60K,0.0,27.0,2.0,1586.0,4350.0,71.0,0.893,0.11
7077,0,0,1,1.0,42.0,1.0,3.0,High School,$40K - $60K,0.0,29.0,3.0,0.0,4739.0,80.0,0.000,1.00
7078,0,0,1,1.0,42.0,1.0,1.0,College,Less than $40K,0.0,22.0,3.0,1105.0,4186.0,77.0,0.460,0.54
7079,0,1,0,1.0,45.0,1.0,4.0,College,Less than $40K,0.0,36.0,1.0,2517.0,4405.0,55.0,0.752,0.25


In [11]:
#use feature creation to make a ratio between transaction amount and trans count
starter_df['AvgTransValue'] = (starter_df['Total_Trans_Amt'] / starter_df['Total_Trans_Ct']).round(2)
starter_df.drop(columns=['Total_Trans_Amt', 'Total_Trans_Ct'], inplace=True)

In [12]:
"""
The education category 'Graduate' is vague/ambiguous 
In 2021, the highest level of education of the population age 25 and older in the United States was distributed as follows: 
    8.9% had less than a high school diploma or equivalent.
        our dataset shows 17% uneducated
    27.9% had high school graduate as their highest level of school completed. 
        our dataset shows 23.3%
    14.9% had completed some college but not a degree.
        ours only has an ambiguous 'graduate' as 36.6%. could be this, could mean post-grad, term is used interchangeably.
    10.5% had an associate degree as their highest level of school completed.
        our dataset has no distinction between associate/undergrad. just says 'college'. ours shows 11.9% for college
    23.5% had a bachelors degree as their highest degree.
        ours shows 11.9% for college
    14.4% had completed an advanced degree such as a masters degree, professional degree or doctoral degree. 
        ours shows 6.1% post-graduate, 5.1% doctorate, and the ambiguous 36.6% 'graduate'ArithmeticError

#This is a quick dataframe to show the value counts and percentages for the Education column
#This should help evaluate the impact of the Graduate column considering its ambiguous classification
counts = starter_df['Education'].value_counts()
percs = starter_df['Education'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.concat([counts,percs], axis=1, keys=['count', 'percentage'])


It's hard to tell what they mean by 'graduate'. Unfortunately, its the bulk of our data, and dropping it will hurt our sample size. 
    The existence of a 'college' column suggests that graduate could mean beyond college. 
        Alternatively, it could mean you attended college but did not graduate, and 'graduate' represents standard undergrads.
Its a high amount of the data (36.6%), so its likely a mix of multiple categories not included, i.e. trade schools, associate degrees, dropouts, specializations
With so little info on what Graduate means, I'm dropping it all. 
We could try to create a weighting system that roughly ranks the other columns based on years at school (?)
    Essentially:
        uneducated = 0, highschool=1, college=2, post-grad=3, doctorate=4

"""
#OPTION1
#Change education to ranking system, drop graduate
#starter_df = starter_df[starter_df.Education != 'Graduate']
#starter_df = starter_df.replace({'Education' : {'Uneducated': 0, 'High School': 1, 'College' : 2, 'Post-Graduate' : 3, 'Doctorate' : 4}})
#starter_df.Education = starter_df.Education.astype(int)

#OPTION2
#Here is another method that highly simplifies the data but includes the Graduate data.
starter_df = starter_df.replace({'Education' : {'Uneducated': 0, 'High School': 1, 'College' : 2, 'Graduate' :  3, 'Post-Graduate' : 4, 'Doctorate' : 5}})
starter_df.Education = starter_df.Education.astype(int)



In [13]:
#use feature creation to make a ratio between age and tenure
starter_df['TenureByAge'] = (starter_df['Tenure(month)'] / starter_df['Age']).round(2)
starter_df.drop(columns=['Tenure(month)', 'Age'], inplace=True)



In [14]:
starter_df

,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Status,Gender,Dependents,Education,Income,Card Type,Months_Inactive_12_mon,Total_Revolving_Bal,Avg_Utilization_Ratio,CreditUsage,AvgTransValue,TenureByAge
0,0,1,0,1.0,0.0,3.0,1,$60K - $80K,0.0,1.0,777.0,0.061,0.94,27.24,0.87
1,0,0,1,1.0,1.0,5.0,3,Less than $40K,0.0,1.0,864.0,0.105,0.90,39.12,0.90
2,0,1,0,1.0,0.0,3.0,3,$80K - $120K,0.0,1.0,0.0,0.000,1.00,94.35,0.71
4,0,1,0,1.0,0.0,3.0,0,$60K - $80K,0.0,1.0,0.0,0.000,1.00,29.14,0.52
5,0,0,1,1.0,0.0,2.0,3,$40K - $60K,0.0,1.0,1247.0,0.311,0.69,45.33,0.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7076,0,0,1,1.0,1.0,2.0,1,$40K - $60K,0.0,2.0,1586.0,0.893,0.11,61.27,0.71
7077,0,0,1,1.0,1.0,3.0,1,$40K - $60K,0.0,3.0,0.0,0.000,1.00,59.24,0.69
7078,0,0,1,1.0,1.0,1.0,2,Less than $40K,0.0,3.0,1105.0,0.460,0.54,54.36,0.52
7079,0,1,0,1.0,1.0,4.0,2,Less than $40K,0.0,1.0,2517.0,0.752,0.25,80.09,0.80


In [15]:
#Rank the income as a 0-4 low-high
starter_df = starter_df.replace({'Income' : {'Less than $40K': 0, '$40K - $60K': 1, '$80K - $120K' : 2, '$60K - $80K' : 3, '$120K +' : 4}})
starter_df.Income = starter_df.Income.astype(int)


In [16]:
#Define features(X) and target(y)
y = starter_df['Status']
X = starter_df.drop(columns=['Status'])

In [17]:
#Split features and target in to training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [18]:
starter_df.dtypes

Marital_Status_Divorced      int32
Marital_Status_Married       int32
Marital_Status_Single        int32
Status                     float64
Gender                     float64
Dependents                 float64
Education                    int32
Income                       int32
Card Type                  float64
Months_Inactive_12_mon     float64
Total_Revolving_Bal        float64
Avg_Utilization_Ratio      float64
CreditUsage                float64
AvgTransValue              float64
TenureByAge                float64
dtype: object

In [19]:
#Scale all numerical/float values that don't represent categories

scaler = StandardScaler()

#Use Column Transformer to scale only the numerical/float values that don't represent categories (male/female, status, )
coltran= ColumnTransformer([
('Total_Revolving_Bal_Scaled', scaler, ['Total_Revolving_Bal']),
('Dependents_Scaled', scaler, ['Dependents']),
('Education_Scaled', scaler, ['Education']),
('Income_Scaled', scaler, ['Income']),
('Card Type Scaled', scaler, ['Card Type']),
('Months_Inactive_12_mon_scaled', scaler, ['Months_Inactive_12_mon']),
('CreditUsageScaled', scaler, ['CreditUsage']),
('AvgTransValueScaled', scaler, ['AvgTransValue']),
('TenureByAgeScaled', scaler, ['TenureByAge']),
('EducationRanks', scaler, ['Education']),
('Avg_Util_Ratio_Scaled', scaler, ['Avg_Utilization_Ratio'])
])

X_train = coltran.fit_transform(X_train)
X_test = coltran.transform(X_test)


In [20]:
#Use SMOTE to add synthetic data and balance our target feature value count
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

#COMMENT OUT THIS WHOLE CELL IF USING BALANCEDRANDOMFORESTCLASSIFIER

In [21]:
#Choose a model

#STILL THE BEST!
#TRY WITH BOTH EDUCATION COLUMN OPTIONS (with and without Graduate data)

clf = GradientBoostingClassifier(
n_estimators=500,    #default = 100    range = 1-inf
random_state=2,     #default = None   range =
subsample=0.9,     #default = 1   range =
min_samples_split = 4,      #default = 2   range = 2-inf
#max_depth=3,  #default = 3    range = 1-inf
#min_impurity_decrease=0,    #default = 0    range = 0 - inf
#min_samples_leaf = 1,            #default = 1   range = 1 - inf
#min_weight_fraction_leaf = 0,     #default = 0   range =0 - 0.5
max_leaf_nodes = 4     #default = None   range = 2-inf
)


'''
clf = XGBClassifier(
n_estimators = 50, 
max_depth = 4, 
objective='binary:logistic'
)
'''


#clf = AdaBoostClassifier(n_estimators=200, random_state=2, learning_rate = 0.2)

#svc = SVC(kernel='linear')
#clf = AdaBoostClassifier(base_estimator=svc, algorithm='SAMME')

#svc = LinearSVC()
#clf = AdaBoostClassifier(base_estimator=svc, algorithm='SAMME')

#svc = SVC()
#clf = AdaBoostClassifier(base_estimator=svc, algorithm='SAMME')

#clf = BalancedRandomForestClassifier()

#clf = SVC()
#clf = SVC(probability=True, kernel='linear')

#clf = LinearSVC()
#clf = LinearSVC(probability=True, kernel='linear')

#clf = LogisticRegression()

#clf = RandomForestClassifier()

"\nclf = XGBClassifier(\nn_estimators = 50, \nmax_depth = 4, \nobjective='binary:logistic'\n)\n"

In [22]:
#Fit the model on the training data
clf.fit(X_train, y_train)

GradientBoostingClassifier(max_leaf_nodes=4, min_samples_split=4,
                           n_estimators=500, random_state=2, subsample=0.9)

In [23]:
#Make predictions on the test data
test_predictions = clf.predict(X_test)

In [24]:
# Evaluate the model's performance
accuracy = accuracy_score(y_test, test_predictions)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 85.24%


In [25]:
#Generate a test matrix
test_matrix = confusion_matrix(y_test, test_predictions)
print(test_matrix)

[[ 89  90]
 [ 95 979]]


In [26]:
# Create a testing classifiction report
testing_report = classification_report(y_test, test_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

        -1.0       0.48      0.50      0.49       179
         1.0       0.92      0.91      0.91      1074

    accuracy                           0.85      1253
   macro avg       0.70      0.70      0.70      1253
weighted avg       0.85      0.85      0.85      1253



In [27]:
'''

BEST PRECISION SO FAR

BALANCED FOREST,ADABOOSTER AND SVC w/ SMOTE SEEM TO RAISE RECALL A LOT BUT NOT PRECISION


*current best
clf = GradientBoostingClassifier(
n_estimators=500,    #default = 100    range =
random_state=2,     #default = None   range =
subsample=0.9,     #default = 1   range =
min_samples_split = 4,      #default = 2   range = 2-inf
max_depth=3,  #default = 3    range = 1-inf
min_impurity_decrease=0,    #default = 0    range = 0 - inf
min_samples_leaf = 1,            #default = 1   range = 1 - inf
min_weight_fraction_leaf = 0,     #default = 0   range =0 - 0.5
max_leaf_nodes = 4     #default = None   range = 2-inf
)
gives = 51/50/50
#similar scores when adding max_features=10 and keeping the rest above the same


clf = GradientBoostingClassifier(
n_estimators=1000, 
random_state=3, 
subsample=0.9, 
min_samples_split = 4, 
max_depth=3, 
min_impurity_decrease=1
) 
GIVES = 42/61/50


clf = GradientBoostingClassifier(
n_estimators=1000,    #default = 100
random_state=3,     #default = None
subsample=0.9,     #default = 1
min_samples_split = 4,      #default = 2
max_depth=3,  #default = 3
min_impurity_decrease=0,    #default = 0
min_samples_leaf = 1,            #default = 1
min_weight_fraction_leaf = 0,     #default = 0
max_leaf_nodes = 4     #default = None
)
GIVES = 52, 43, 47


clf = GradientBoostingClassifier(
n_estimators=1000, 
random_state=3, 
subsample=0.9, 
min_samples_split = 4
)
GIVES = 50/43/46





'''

'\n\nBEST PRECISION SO FAR\n\nBALANCED FOREST,ADABOOSTER AND SVC w/ SMOTE SEEM TO RAISE RECALL A LOT BUT NOT PRECISION\n\n\n*current best\nclf = GradientBoostingClassifier(\nn_estimators=500,    #default = 100    range =\nrandom_state=2,     #default = None   range =\nsubsample=0.9,     #default = 1   range =\nmin_samples_split = 4,      #default = 2   range = 2-inf\nmax_depth=3,  #default = 3    range = 1-inf\nmin_impurity_decrease=0,    #default = 0    range = 0 - inf\nmin_samples_leaf = 1,            #default = 1   range = 1 - inf\nmin_weight_fraction_leaf = 0,     #default = 0   range =0 - 0.5\nmax_leaf_nodes = 4     #default = None   range = 2-inf\n)\ngives = 51/50/50\n#similar scores when adding max_features=10 and keeping the rest above the same\n\n\nclf = GradientBoostingClassifier(\nn_estimators=1000, \nrandom_state=3, \nsubsample=0.9, \nmin_samples_split = 4, \nmax_depth=3, \nmin_impurity_decrease=1\n) \nGIVES = 42/61/50\n\n\nclf = GradientBoostingClassifier(\nn_estimators=10